In [ ]:
# !/bin/bash
curl -L -o /brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce -o  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

https://www.docstomarkdown.pro/convert-word-to-markdown/

  

  

1\. Deploy a spark Cluster ( like AWS EMR, HDinsight or dataproc or a company internal )

  

2\. Store Data in HDFS

  

a. **Curl Command** - # !/bin/bash curl -L -o /brazilian-ecommerce.zip\\ https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce -o https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  

2\. Unzip the file :- unzip brazilian-ecommerce.zip

  

3\. Send to hadoop.

  

hadoop fs -mkdir /olist\_ka\_data

  

hadoop fs -put \*.csv /olist\_ka\_data

  

  

4\. Use Pyspark on Cluster to work on data.

# No Class Tomorrow

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

25/03/02 04:43:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark

In [5]:
!hadoop fs -ls /olist_ka_data/

Found 9 items
-rw-r--r--   2 root hadoop    9033957 2025-03-02 04:36 /olist_ka_data/olist_customers_dataset.csv
-rw-r--r--   2 root hadoop   61273883 2025-03-02 04:36 /olist_ka_data/olist_geolocation_dataset.csv
-rw-r--r--   2 root hadoop   15438671 2025-03-02 04:36 /olist_ka_data/olist_order_items_dataset.csv
-rw-r--r--   2 root hadoop    5777138 2025-03-02 04:36 /olist_ka_data/olist_order_payments_dataset.csv
-rw-r--r--   2 root hadoop   14451670 2025-03-02 04:36 /olist_ka_data/olist_order_reviews_dataset.csv
-rw-r--r--   2 root hadoop   17654914 2025-03-02 04:36 /olist_ka_data/olist_orders_dataset.csv
-rw-r--r--   2 root hadoop    2379446 2025-03-02 04:36 /olist_ka_data/olist_products_dataset.csv
-rw-r--r--   2 root hadoop     174703 2025-03-02 04:36 /olist_ka_data/olist_sellers_dataset.csv
-rw-r--r--   2 root hadoop       2613 2025-03-02 04:36 /olist_ka_data/product_category_name_translation.csv


In [7]:
hdfs_path = '/olist_ka_data/'

In [8]:
customer_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv')

In [9]:
customer_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------+
|                 _c0|                 _c1|                 _c2|                 _c3|           _c4|
+--------------------+--------------------+--------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code...|       customer_city|customer_state|
|06b8999e2fba1a1fb...|861eff4711a542e4b...|               14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|               09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|               01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|               08775|     mogi das cruzes|            SP|
+--------------------+--------------------+--------------------+--------------------+--------------+
only showing top 5 rows



In [10]:
olist_customers_df = spark.read.csv("/olist_ka_data/olist_customers_dataset.csv", header=True, inferSchema=True)
olist_geolocation_df = spark.read.csv("/olist_ka_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)
olist_order_items_df = spark.read.csv("/olist_ka_data/olist_order_items_dataset.csv", header=True, inferSchema=True)
olist_order_payments_df = spark.read.csv("/olist_ka_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
olist_order_reviews_df = spark.read.csv("/olist_ka_data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
olist_orders_df = spark.read.csv("/olist_ka_data/olist_orders_dataset.csv", header=True, inferSchema=True)
olist_products_df = spark.read.csv("/olist_ka_data/olist_products_dataset.csv", header=True, inferSchema=True)
olist_sellers_df = spark.read.csv("/olist_ka_data/olist_sellers_dataset.csv", header=True, inferSchema=True)
product_category_name_translation_df = spark.read.csv("/olist_ka_data/product_category_name_translation.csv", header=True, inferSchema=True)


In [11]:
olist_customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [12]:
olist_order_items_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [15]:
# Data Leakage

print(f'Customers : {olist_customers_df.count()} rows')

Customers : 99441 rows


In [16]:
print(f'Orders : {olist_orders_df.count()} rows')

Orders : 99441 rows


# Do for all other Datasets

In [21]:
olist_customers_df.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

# Count/ Check of Null

In [19]:
from pyspark.sql.functions import *

# unnecessary imports make things slower to boot up and testing would take longer

In [23]:
olist_customers_df.select([count(when(col(c).isNull(),1)).alias(c) \
                          for c in olist_customers_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



So what's the idea for ingestion? I am guessing - for warehousing purposes you will try to denormalize data and fit into start schema after analyzing ER diagram?

Can't we use describe function to find the number of missing rows for all the columns in dataframe?

In [26]:
# Duplicate Values

olist_customers_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



# Data Exploration

In [28]:
olist_customers_df.groupBy('customer_state').count().orderBy('count',ascending=False).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [29]:
olist_orders_df.groupBy('order_status').count().orderBy('count',ascending=False).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [30]:
olist_orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [32]:
olist_order_payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

just from seeing the kaggle dataset it's difficult to understand issues with the data so anything you can suggest to fill the gaps incase we don't have the appropriate domain knowledge

In [33]:
olist_order_payments_df.groupBy('payment_type').count().orderBy('count',ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+

